In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load IMDB dataset from Keras, keeping only the top 10,000 most frequent words
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

# Pad sequences to ensure uniform length for all reviews
max_len_short = 200  # Max length for short sequences (IMDB reviews)
X_train_padded = pad_sequences(X_train, maxlen=max_len_short)
X_test_padded = pad_sequences(X_test, maxlen=max_len_short)

print(f"X_train_padded shape: {X_train_padded.shape}, X_test_padded shape: {X_test_padded.shape}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X_train_padded shape: (25000, 200), X_test_padded shape: (25000, 200)


In [3]:
# Simulate long sequences (e.g., Wikipedia-like data with 1500 tokens per sequence)
long_sequences = np.random.randint(10000, size=(1000, 1500))  # 1000 samples, each with 1500 tokens
labels_long = np.random.randint(2, size=(1000,))  # Binary labels for the simulated long sequences

# Split long sequence dataset into training and testing sets
X_train_long, X_test_long, y_train_long, y_test_long = train_test_split(long_sequences, labels_long, test_size=0.2)

# Pad long sequences to a fixed length of 500 tokens
max_len_long = 500  # Truncate/pad sequences to 500 tokens
X_train_long_padded = pad_sequences(X_train_long, maxlen=max_len_long)
X_test_long_padded = pad_sequences(X_test_long, maxlen=max_len_long)

print(f"X_train_long_padded shape: {X_train_long_padded.shape}, X_test_long_padded shape: {X_test_long_padded.shape}")

X_train_long_padded shape: (800, 500), X_test_long_padded shape: (200, 500)


In [4]:
# Function to build a Sequential RNN model
def build_model_rnn(input_length, units=64, num_layers=1):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=input_length))
    for i in range(num_layers):
        if i == num_layers - 1:
            model.add(SimpleRNN(units, return_sequences=False))
        else:
            model.add(SimpleRNN(units, return_sequences=True))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to build a Sequential LSTM model
def build_model_lstm(input_length, units=64, num_layers=1):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=128, input_length=input_length))
    for i in range(num_layers):
        if i == num_layers - 1:
            model.add(LSTM(units, return_sequences=False))
        else:
            model.add(LSTM(units, return_sequences=True))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# Function to train and evaluate the model
def train_and_evaluate(X_train, X_test, y_train, y_test, units=64, num_layers=1, max_len=200, model_type='RNN'):
    if model_type == 'RNN':
        model = build_model_rnn(input_length=max_len, units=units, num_layers=num_layers)
    else:
        model = build_model_lstm(input_length=max_len, units=units, num_layers=num_layers)

    model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2, verbose=1)

    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).flatten()  # Convert predictions to binary labels

    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_type} - Units: {units}, Layers: {num_layers}, Accuracy: {accuracy:.4f}")

    return accuracy

In [6]:
# Evaluate RNNs on the IMDB dataset (short sequences)
print("Short Sequence Dataset (IMDB)")
short_seq_results = []

# RNN with 1 layer, 64 units
short_seq_results.append(train_and_evaluate(X_train_padded, X_test_padded, y_train, y_test, units=64, num_layers=1, max_len=max_len_short))

# RNN with 2 layers, 64 units
short_seq_results.append(train_and_evaluate(X_train_padded, X_test_padded, y_train, y_test, units=64, num_layers=2, max_len=max_len_short))

# RNN with 1 layer, 128 units
short_seq_results.append(train_and_evaluate(X_train_padded, X_test_padded, y_train, y_test, units=128, num_layers=1, max_len=max_len_short))

# RNN with 2 layers, 128 units
short_seq_results.append(train_and_evaluate(X_train_padded, X_test_padded, y_train, y_test, units=128, num_layers=2, max_len=max_len_short))

Short Sequence Dataset (IMDB)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 112ms/step - accuracy: 0.6036 - loss: 0.6371 - val_accuracy: 0.8122 - val_loss: 0.4159
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 101ms/step - accuracy: 0.8347 - loss: 0.3872 - val_accuracy: 0.8014 - val_loss: 0.4563
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 100ms/step - accuracy: 0.9030 - loss: 0.2506 - val_accuracy: 0.8300 - val_loss: 0.4583
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 42s 102ms/step - accuracy: 0.9819 - loss: 0.0677 - val_accuracy: 0.8160 - val_loss: 0.5675
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 108ms/step - accuracy: 0.9844 - loss: 0.0488 - val_accuracy: 0.7870 - val_loss: 0.6860
782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step
RNN - Units: 64, Layers: 1, Accuracy: 0.7844
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 57s 167ms/step - accuracy: 0.5698 - loss: 0.6641 - val_accuracy: 0.7328 - val_loss: 0.5362
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 168ms/step - accuracy: 0.8563 - loss: 0.3421 - val_accuracy: 0.8284 - val_loss: 0.4291
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 51s 164ms/step - accuracy: 0.9622 - loss: 0.1126 - val_accuracy: 0.8216 - val_loss: 0.5167
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 57s 182ms/step - accuracy: 0.9817 - loss: 0.0594 - val_accuracy: 0.7528 - val_loss: 0.8078
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 77s 168ms/step - accuracy: 0.9914 - loss: 0.0258 - val_accuracy: 0.7878 - val_loss: 0.8713
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 34ms/step
RNN - Units: 64, Layers: 2, Accuracy: 0.7758
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 47s 144ms/step - accuracy: 0.5707 - loss: 0.6634 - val_accuracy: 0.6918 - val_loss: 0.5765
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 144ms/step - accuracy: 0.7855 - loss: 0.4840 - val_accuracy: 0.7886 - val_loss: 0.4603
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 45s 143ms/step - accuracy: 0.6699 - loss: 0.6580 - val_accuracy: 0.7144 - val_loss: 0.5546
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 142ms/step - accuracy: 0.7789 - loss: 0.4693 - val_accuracy: 0.7240 - val_loss: 0.5630
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 46s 146ms/step - accuracy: 0.7858 - loss: 0.4545 - val_accuracy: 0.5894 - val_loss: 0.6526
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step
RNN - Units: 128, Layers: 1, Accuracy: 0.5965
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


313/313 ━━━━━━━━━━━━━━━━━━━━ 94s 290ms/step - accuracy: 0.5279 - loss: 0.6934 - val_accuracy: 0.5472 - val_loss: 0.7466
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 89s 284ms/step - accuracy: 0.7026 - loss: 0.5645 - val_accuracy: 0.7586 - val_loss: 0.5277
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 141s 281ms/step - accuracy: 0.7767 - loss: 0.4584 - val_accuracy: 0.6616 - val_loss: 0.6286
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 145s 291ms/step - accuracy: 0.8193 - loss: 0.4006 - val_accuracy: 0.7270 - val_loss: 0.6295
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 85s 272ms/step - accuracy: 0.8801 - loss: 0.2953 - val_accuracy: 0.7442 - val_loss: 0.6170
782/782 ━━━━━━━━━━━━━━━━━━━━ 39s 49ms/step
RNN - Units: 128, Layers: 2, Accuracy: 0.7482


In [7]:
# Evaluate RNNs on the simulated long sequences (e.g., Wikipedia-like)
print("\nLong Sequence Dataset (Simulated Wikipedia)")
long_seq_results = []

# RNN with 1 layer, 64 units
long_seq_results.append(train_and_evaluate(X_train_long_padded, X_test_long_padded, y_train_long, y_test_long, units=64, num_layers=1, max_len=max_len_long))

# RNN with 2 layers, 64 units
long_seq_results.append(train_and_evaluate(X_train_long_padded, X_test_long_padded, y_train_long, y_test_long, units=64, num_layers=2, max_len=max_len_long))

# RNN with 1 layer, 128 units
long_seq_results.append(train_and_evaluate(X_train_long_padded, X_test_long_padded, y_train_long, y_test_long, units=128, num_layers=1, max_len=max_len_long))

# RNN with 2 layers, 128 units
long_seq_results.append(train_and_evaluate(X_train_long_padded, X_test_long_padded, y_train_long, y_test_long, units=128, num_layers=2, max_len=max_len_long))


Long Sequence Dataset (Simulated Wikipedia)
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 239ms/step - accuracy: 0.5397 - loss: 0.6982 - val_accuracy: 0.5375 - val_loss: 0.7280
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 202ms/step - accuracy: 0.8242 - loss: 0.5400 - val_accuracy: 0.4500 - val_loss: 0.7031
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 253ms/step - accuracy: 1.0000 - loss: 0.2986 - val_accuracy: 0.4688 - val_loss: 0.7118
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - accuracy: 1.0000 - loss: 0.1318 - val_accuracy: 0.4812 - val_loss: 0.7300
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 392ms/step - accuracy: 1.0000 - loss: 0.0568 - val_accuracy: 0.4750 - val_loss: 0.7576
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
RNN - Units: 64, Layers: 1, Accuracy: 0.5450
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 418ms/step - accuracy: 0.5018 - loss: 0.7180 - val_accuracy: 0.4812 - val_loss: 0.7118
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 542ms/step - accuracy: 0.9254 - loss: 0.3994 - val_accuracy: 0.5063 - val_loss: 0.7984
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 353ms/step - accuracy: 1.0000 - loss: 0.0655 - val_accuracy: 0.4938 - val_loss: 0.9215
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 366ms/step - accuracy: 1.0000 - loss: 0.0102 - val_accuracy: 0.5000 - val_loss: 1.0443
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 390ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.5000 - val_loss: 1.0818


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 336ms/step - accuracy: 0.5035 - loss: 0.7316 - val_accuracy: 0.4812 - val_loss: 0.7341
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 468ms/step - accuracy: 0.6080 - loss: 0.6420 - val_accuracy: 0.4500 - val_loss: 0.7126
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 325ms/step - accuracy: 0.7704 - loss: 0.5747 - val_accuracy: 0.5312 - val_loss: 0.6863
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 372ms/step - accuracy: 0.9920 - loss: 0.4443 - val_accuracy: 0.5125 - val_loss: 0.7017
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 621ms/step - accuracy: 0.9950 - loss: 0.2436 - val_accuracy: 0.3875 - val_loss: 0.8404


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step
RNN - Units: 128, Layers: 1, Accuracy: 0.5600
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 824ms/step - accuracy: 0.5098 - loss: 0.7370 - val_accuracy: 0.5500 - val_loss: 0.6933
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 570ms/step - accuracy: 0.7950 - loss: 0.4743 - val_accuracy: 0.4938 - val_loss: 0.8232
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 561ms/step - accuracy: 1.0000 - loss: 0.0399 - val_accuracy: 0.5375 - val_loss: 1.1712
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 610ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.5750 - val_loss: 1.3484
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 649ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.5188 - val_loss: 1.4779
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step
RNN - Units: 128, Layers: 2, Accuracy: 0.5950
